<h1> Kjeppevikholmen Optical Analysis -- June Growth Trend </h1>

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from aquabyte.visualize import Visualizer
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
import json, os
import cv2
import torch
import pytz
from PIL import Image
import datetime as dt
import dateutil


In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
query = """
    select * from keypoint_annotations
    where pen_id=5
    and keypoints -> 'leftCrop' is not null
    and keypoints -> 'rightCrop' is not null
    and is_qa=FALSE;
"""
df = rds_access_utils.extract_from_database(query)

In [ ]:
normalize_centered_2D_transform_biomass = NormalizeCentered2D()
normalized_stability_transform = NormalizedStabilityTransform()
to_tensor_transform = ToTensor()

# load neural network weights
biomass_network = torch.load('/root/data/alok/biomass_estimation/results/neural_network/2019-11-08T00:13:09/nn_epoch_798.pb')

In [ ]:
def generate_weight(row_id, keypoints, cm):
    
    # run biomass estimation
    input_sample = {
        'keypoints': keypoints,
        'cm': cm,
        'stereo_pair_id': row_id,
        'single_point_inference': True
    }
    nomralized_centered_2D_kps = \
        normalize_centered_2D_transform_biomass.__call__(input_sample)

    normalized_stability_kps = normalized_stability_transform.__call__(nomralized_centered_2D_kps)
    tensorized_kps = to_tensor_transform.__call__(normalized_stability_kps)
    weight_prediction = biomass_network(tensorized_kps['kp_input']).item() * 1e4
    
    return weight_prediction

In [ ]:
weights = []
count = 0
for idx, row in df.iterrows():
    if count % 1000 == 0:
        print(count)
    count += 1
    weight = generate_weight(row.id, row.keypoints, row.camera_metadata)
    weights.append(weight)


In [ ]:
df['weight'] = weights
# df = df.sort_values('captured_at', ascending=True)
 

In [ ]:
df.index = pd.to_datetime(df.captured_at)


In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(df.weight.rolling('5D').mean())
plt.grid()
plt.show()

In [ ]:
tdf = df.weight.resample('D', how=lambda x: np.mean(x))
for d in range(1, 6):
    fig, ax = plt.subplots(figsize=(10, 5))
    kdf = tdf.rolling(d).mean()
    ax.scatter(kdf.index, kdf.values)
    ax.set_title('Blom Kjeppevikholmen pen_id=5 (Pen #3): {} day aggregation'.format(d))
    ax.set_xlabel('Date')
    ax.set_ylabel('Predicted Weight (g)')
    ax.grid()
    fig.autofmt_xdate()
    plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(df[df.index > '2019-06-27'].weight, bins=100)
plt.title('Predicted weight distribution for 6/27 onward')
plt.xlabel('Predicted weight (g)')
plt.ylabel('Frequency')
plt.grid()
plt.show()

<h1> Optical Sampling Bias Study </h1>

<h2> Simulate FOVs of all sizes, and for each one get the average crop width / crop height / crop area. For FOVs smaller than the current one, get the average bimoass as well </h2>

In [ ]:
# Study 1: Crop width / height / area study

fovs = list(np.arange(1, 60, 1))




In [ ]:
fov = 30
pct_coverage = 